# SCC0230 - Inteligência Artificial (2030)
#### Docente: Solange Oliveira Rezende
#### PAE - Germano Antonio Zani Jorge
#### PAE - Rene Vieira Santin
### **Trabalho 1**
#### Integrantes:
*  Rafael Corona - 4769989
*  Vitor Caetano Brustolin - 11795589
*  Rafael Corona - 4769989
*  Rafael Corona - 4769989
*  Rafael Corona - 4769989

### **PARTE 1**

Colocar aqui:
*  Qual a motivação por traz desse trabalho
*  Qual o problema
*  Por que escolhemos esse problema
*  Quais são as variaveis
*  Quais são os parâmetros
*  Qual o Nó inicial (Podemos escolher qualquer um)
*  Explicar como modelamos o problema (falar um pouquinho da classe que utilizamos para implementar a parada
)

### Imports

In [37]:
import itertools
import time
import numpy as np

In [43]:
#classe que representa um grafo valorado não orientado
class WeightedGraph:
    def __init__(self):
        self.graph = {} #listas de adjacencia
        self.N_vertices = 0
        self.N_edges = 0

    #adiciona um vértice ao grafo, caso ele ainda não tenha sido adicionado
    def add_vertex(self, vertex):
        if (vertex not in self.graph):
            self.graph[vertex] = {} #lista de adjacencia do vétice
            self.N_vertices = self.N_vertices+1

    #adiciona uma aresta entre dois vértices do grafo
    def add_edge(self, vertex1, vertex2, weight):
        if (vertex1 in self.graph and vertex2 in self.graph):
            if(weight >= 0):
                self.graph[vertex1][vertex2] = weight
                self.graph[vertex2][vertex1] = weight  # para um grafo não direcionado
            
            #distâncias negativas significam que não há caminho
            else:
                self.graph[vertex1][vertex2] = float('inf')
                self.graph[vertex2][vertex1] = float('inf')
            self.N_edges = self.N_edges+1
    
    #remove uma aresta entre dois vértices do grafo
    def remove_edge(self, vertex1, vertex2):
        if (vertex1 in self.graph and vertex2 in self.graph):
            del self.graph[vertex1][vertex2]
            del self.graph[vertex2][vertex1]# para um grafo não direcionado
            self.N_edges = self.N_edges-1 

    #printa o grafo como uma lista de adjacencias
    def print(self):
        for vertex, neighbors in self.graph.items():
            neighbor_str = ", ".join([f"{neighbor} ({weight})" for neighbor, weight in neighbors.items()])
            print(f"{vertex}: {neighbor_str}")

    #lê um grafo a partir de um arquivo
    def read_from_file(self, file_path):
            graph = self()
            with open(file_path, 'r') as file:
                num_vertices = int(file.readline().strip())
                num_edges = int(file.readline().strip())

                for _ in range(num_vertices):
                    vertex = file.readline().strip()
                    graph.add_vertex(vertex)

                for _ in range(num_edges):
                    line = file.readline().strip()
                    vertex1, vertex2, weight = line.split()
                    vertex1 = str(vertex1)
                    vertex2 = str(vertex2)
                    weight = float(weight)
                    graph.add_edge(vertex1, vertex2, weight)
                #linhas após o termino das arestas são ignoradas

    def convert_to_matrix(self):
        vertices = list(self.graph.keys())
        matrix = []
        for v in vertices:
            matrix.append([])
            for w in range(self.N_vertices):
                matrix[v][w] = self.graph[v][w]
                
    

    #resolve o problema do caixeiro viajante por força bruta
    #retorna o menor caminho e o comprimento desse caminho
    def tsp_bruteforce(self):
        vertices = list(self.graph.keys())

        shortest_path = []
        shortest_distance = float('inf')

        if self.N_vertices < 2:
            return shortest_path, 0  #um único vértice não configura caminho 

        #lista cada permutação possível dos vértices
        for permuted_vertices in itertools.permutations(vertices):
            total_distance = 0
            for i in range(self.N_vertices - 1):
                vertex1 = permuted_vertices[i]
                vertex2 = permuted_vertices[i + 1]
                total_distance += self.graph[vertex1][vertex2]

            # soma as distancias do primeiro ao ultimo vertice
            total_distance += self.graph[permuted_vertices[-1]][permuted_vertices[0]]

            if total_distance < shortest_distance:#escolhe a menor
                shortest_distance = total_distance
                shortest_path = list(permuted_vertices)

        return shortest_path, shortest_distance
    

    def tsp_best_first(self, start_vertex=None):
        if not start_vertex:
            start_vertex = next(iter(self.graph))  # Comecamos do primeiro vertice

        unvisited = set(self.graph.keys()) 
        unvisited.discard(start_vertex) #visitamos o primeiro vertice
        tour = [start_vertex]
        current_vertex = start_vertex

        while unvisited:
            closest_neighbor = None
            min_distance = float('inf')

            for neighbor, weight in self.graph[current_vertex].items():
                if neighbor in unvisited and weight < min_distance:
                    min_distance = weight
                    closest_neighbor = neighbor

            if closest_neighbor is not None:
                tour.append(closest_neighbor)
                unvisited.remove(closest_neighbor)
                current_vertex = closest_neighbor
            else:
                # Caso todos os visinhos ja foram visitados
                # Não ocorre no nosso problema
                current_vertex = tour[0]

        # Soma as distancias
        tour_distance = sum(self.graph[tour[i]][tour[i + 1]] for i in range(len(tour) - 1))
        tour_distance += self.graph[tour[-1]][tour[0]]

        return tour, tour_distance
    
    def tsp_branch_bound(self, distances):
        n = self.N_vertices
        best_path = None
        best_cost = np.inf
            
        # inicializaremos a stack com o no raiz
        stack = [(0, [0], set([0]), 0)]
        while stack:
            node = stack.pop()
            if len(node[1]) == n:
                # se todas os nos foram visitados, atualize o melhor caminho se o caminho novo eh mais curto
                cost = node[3] + distances[int(node[1][-1])][int(0)]
                if cost < best_cost:
                    best_path = node[1]
                    best_cost = cost
            else:
                # gere nos criancas considerando todos os nos nao visitados
                unvisited = set(range(n)) - node[2]
                for i in unvisited:
                    child_path = node[1] + [i]
                    child_cost = node[3] + distances[int(node[1][-1])][int(i)]
                    # podar nos com custo maior do que o nosso melhor custo atual
                    if child_cost < best_cost:
                        stack.append((i, child_path, node[2] | set([i]), child_cost))
        return best_path, best_cost

    # funcao usada para converter a nossa representacao usando listas para uma matrix
    # a fim de facilitar o uso do algoritmo de branch and bound
    def convert_to_matrix(self, matrix):
        vertices = list(self.graph.keys())
        for j in range(self.N_vertices):
            matrix.append([])
            for i in range(self.N_vertices):
                if i != j:
                    vertice1 = vertices[j]
                    vertice2 = vertices[i]
                    matrix[j].append(self.graph[vertice1][vertice2])
                else:
                    matrix[j].append(0)         

In [39]:
graph = WeightedGraph()

PATH = "data/euclidian.txt"

graph.read_from_file(PATH)

graph.print()

ME: VE (10.58), TE (19.4), MA (31.02), JU (117.3), SA (239.2), UR (465.6), NE (729.1), PL (983.2), SO (8.424)
VE: ME (10.58), TE (12.68), MA (25.3), JU (114.6), SA (234.2), UR (458.6), NE (720.7), PL (975.3), SO (12.68)
TE: ME (19.4), VE (12.68), MA (12.68), JU (102.1), SA (221.6), UR (446.6), NE (709.7), PL (963.9), SO (24.71)
MA: ME (31.02), VE (25.3), TE (12.68), JU (89.76), SA (208.9), UR (434.6), NE (698.8), PL (952.5), SO (37.18)
JU: ME (117.3), VE (114.6), TE (102.1), MA (89.76), SA (127.2), UR (361.8), NE (638.1), PL (885.5), SO (125.0)
SA: ME (239.2), VE (234.2), TE (221.6), MA (208.9), JU (127.2), UR (236.6), NE (519.3), PL (761.6), SO (246.0)
UR: ME (465.6), VE (458.6), TE (446.6), MA (434.6), JU (361.8), SA (236.6), NE (290.1), PL (525.1), SO (471.2)
NE: ME (729.1), VE (720.7), TE (709.7), MA (698.8), JU (638.1), SA (519.3), UR (290.1), PL (262.6), SO (733.3)
PL: ME (983.2), VE (975.3), TE (963.9), MA (952.5), JU (885.5), SA (761.6), UR (525.1), NE (262.6), SO (988.0)
SO: M

### **PARTE 2**

Primeiramente, vamos executar a solução força bruta para obtermos alguns parâmetros iniciais, como a solução ótima e o pior tempo de execução possível 

In [40]:
N_EXECUCOES = 20

start = time.time()
for i in range(N_EXECUCOES):
    shortest_path_bruteforce, shortest_distance_bruteforce = graph.tsp_bruteforce()
end = time.time()

print("Caminho mais curto:", shortest_path_bruteforce)
print("Comprimento total:", shortest_distance_bruteforce, "metros")
print("Tempo Médio de execução:", (end-start)/N_EXECUCOES,"segundos")

Caminho mais curto: ['ME', 'SO', 'VE', 'TE', 'MA', 'NE', 'PL', 'UR', 'SA', 'JU']
Comprimento total: 2014.0639999999999 metros
Tempo Médio de execução: 3.2951056122779847 segundos


## **Parte 3**

Agora vamos executar os demais algoritmos e aferir suas estatísticas em relação a implementação força bruta. Primeiramente testando com o best first.

In [41]:
start = time.time()
for i in range(N_EXECUCOES):
    shortest_path_bestfirst, shortest_distance_best_first = graph.tsp_best_first()
end = time.time()

print("Caminho mais curto:", shortest_path_bestfirst)
print("Comprimento total:", shortest_distance_best_first, "metros")
print("Tempo Médio de execução:", (end-start)/N_EXECUCOES,"segundos")
print("Erro relativo da solução:", (shortest_distance_best_first-shortest_distance_bruteforce)/shortest_distance_bruteforce*100)

Caminho mais curto: ['ME', 'SO', 'VE', 'TE', 'MA', 'JU', 'SA', 'UR', 'NE', 'PL']
Comprimento total: 2035.9240000000002 metros
Tempo Médio de execução: 1.2445449829101563e-05 segundos
Erro relativo da solução: 1.0853676943731856


Depois testemos o algoritmo de Branch and Bound.

In [44]:
matrix = []
graph.convert_to_matrix(matrix)
start = time.time()
for i in range(N_EXECUCOES):
    shortest_path_branch_bound, shortest_distance_branch_bound = graph.tsp_branch_bound(matrix)
end = time.time()

print("Caminho mais curto:", shortest_path_branch_bound)
print("Comprimento total:", shortest_distance_branch_bound, "metros")
print("Tempo Médio de execução:", (end-start)/N_EXECUCOES)
print("Erro relativo da solução:", (shortest_distance_branch_bound-shortest_distance_bruteforce)/shortest_distance_bruteforce*100)

Caminho mais curto: [0, 9, 1, 2, 3, 7, 8, 6, 5, 4]
Comprimento total: 2014.0639999999999 metros
Tempo Médio de execução: 0.1155349612236023
Erro relativo da solução: 0.0
